# Тестовое задание SMLab

## Задание 1. SQL.

Дана таблица с позициями чеков checks и с каталогом cat. Посчитать количество чеков, в которых куплено больше 2-х пар любой обуви и есть любой товар из категории туризм.
Написать одним запросом на Oracle SQL.

In [1]:
import sqlite3
import pandas as pd

In [2]:
# создаём (или открываем) базу
conn = sqlite3.connect("test_SM.db")
cur = conn.cursor()

# создаём таблицу cat
cur.execute("""
    CREATE TABLE cat (
        art VARCHAR2(10) PRIMARY KEY,
        product VARCHAR2(30),
        category VARCHAR2(30))
    """)

# создаём таблицу checks
cur.execute("""
    CREATE TABLE checks (
        id_check NUMBER(10),
        id_pos NUMBER(10),
        art VARCHAR2(10),
        quantity NUMBER(10),
        CONSTRAINT PK_KEY_CHECKS PRIMARY KEY (id_check, id_pos),
        CONSTRAINT FK_KEY_CHECKS FOREIGN KEY (art) REFERENCES checks(art))
    """)

# данные для cat
cat_data = [
    ["A1", "кроссовки", "обувь"],
    ["A2", "кеды", "обувь"],
    ["A3", "сланцы", "обувь"],
    ["A4", "куртка", "одежда"],
    ["A5", "ветровка", "одежда"],
    ["A6", "шорты", "одежда"],
    ["A7", "палатка", "туризм"],
    ["A8", "коврик", "туризм"],
    ["A9", "спальник", "туризм"],
    ["A10", "рюкзак", "туризм"],
]

# данные для checks
checks_data = [
    [1, 1, "A1", 1],
    [1, 2, "A4", 3],
    [1, 3, "A2", 2],
    [1, 4, "A10", 1],
    [1, 5, "A8", 4],
    [2, 1, "A5", 1],
    [2, 2, "A7", 1],
    [2, 3, "A3", 2],
    [3, 1, "A2", 1],
    [3, 2, "A9", 1],
    [3, 3, "A5", 2],
    [3, 4, "A6", 1],
    [3, 5, "A3", 3],
    [4, 1, "A1", 3],
    [4, 2, "A2", 1],
    [5, 1, "A2", 1],
    [5, 2, "A8", 5],
    [5, 3, "A3", 1],
    [5, 4, "A4", 1],
    [5, 5, "A5", 1],
]

# вставляем данные
cur.executemany('INSERT OR REPLACE INTO checks VALUES (?, ?, ?, ?)', checks_data)
cur.executemany('INSERT OR REPLACE INTO cat VALUES (?, ?, ?)', cat_data)

conn.commit()
conn.close()

In [3]:
text = """
    SELECT COUNT(*) AS checks_cnt
    FROM (
        SELECT ch.id_check AS id_check
        FROM checks ch JOIN cat c ON ch.art = c.art
        GROUP BY ch.id_check
        HAVING SUM(CASE WHEN c.category = "обувь" THEN ch.quantity ELSE 0 END) > 2 AND
            SUM(CASE WHEN c.category = "туризм" THEN 1 ELSE 0 END) > 0
    )
"""
conn = sqlite3.connect("test_SM.db")
cur = conn.cursor()

cur.execute(text)
res = cur.fetchall()
res = pd.DataFrame(res)
res.columns = ["checks_cnt"]
res

,checks_cnt
0,2


## Задание 2. Python

Написать функцию split_list, которая::
1) принимает на вход итерируемый объект (list, tuple, np.array и т.п.) и число N - кол-во частей, на которые нужно разделить исходный итерируемый объект;

2) возвращает на выходе список из N-частей исходного итерируемого объекта.

Ограничения на результат:

1) Типы элементов, которые хранится внутри исходного итерируемого объекта, не должны измениться в результирующем списке:

[[1,'a','b'], [2, 'c', 'd'], [3, 'e', 'f']], N=2 -> [[[1, 'a', 'b'], [2, 'c', 'd']], [[3, 'e', 'f']]] - хорошо

[[1,'a','b'], [2, 'c', 'd'], [3, 'e', 'f']], N=2 -> [[['1', 'a', 'b'], ['2', 'c', 'd']], [['3', 'e', 'f']]] - плохо

2) Элементы исходного итерируемого объекта должны быть максимально равномерно распределены между частями:

[1,2,3,4,5,6,7,8,9], N=4 -> [[1,2,3], [4,5], [6,7], [8,9]] - хорошо

[1,2,3,4,5,6,7,8,9], N=4 -> [[1], [2,3], [4,5,6], [7,8,9]] - плохо

[1,2,3,4,5,6,7,8,9, 10], N=4 -> [[1,2,3], [4,5, 6], [7,8], [9, 10]] - хорошо

[1,2,3,4,5,6,7,8,9, 10], N=4 -> [[1,2,3], [4,5, 6], [7,8,9], [10]] - плохо

3) Разбиение должно сохранить порядок следования элементов в исходном списке

[1,2,3,4,5,6,7,8,9], N=4 -> [[1,2,3], [4,5], [6,7], [8,9]] - хорошо

[1,2,3,4,5,6,7,8,9], N=4 -> [[1,4,8], [2,5], [3,6], [7,9]] - плохо

4) Части, на которые разобьется итерируемый объект, должны быть расположены в порядке убывания кол-ва элементов внутри этих частей.

[1,2,3,4,5,6,7,8,9], N=4 -> [[1,2,3], [4,5], [6,7], [8,9]] - хорошо

[1,2,3,4,5,6,7,8,9], N=4 -> [[1,2], [3,4], [5,6], [7,8,9]] - плохо


In [40]:
import math
import numpy as np
from typing import Iterable


def split_list(input_obj: Iterable, n: int) -> list:
    """Функция для разбиения итерируемого объекта на N частей.
    Суть в том, чтобы определить для каждой группы из N подходящее кол-во элементов из исходного итерируемого объекта.
    1. Cами объекты никак не преобразуются, что гарантирует выполнение условия 1.
    2. Равномерность распределения размеров групп выполняется, так как задействуется минимально 
        возможное уникальное число размеров групп, и разница размеров между соседними группами не более 1.
    3. Порядок следования элементов исходного объекта сохряняется, 
        так как алгоритм забирает элементы из исходного объекта с начала, а присоединяет к концу нового (своего рода 'очередь').
    4. Количество элементов будет в порядке убывания, так как при определении размеров групп значение округляется в большую сторону ``math.ceil``.

    Parameters
    ----------
    input_obj : Iterable
        Итерируемый объект, элементы которого нужно разбить на группы
    n : int
        Количество групп

    Returns
    -------
    list
        Список, содержащий группы разбиения

    """
    assert len(input_obj) > 0, "Исходный объект пустой"
    assert n > 0, "Количество групп не может быть отрицательным числом или 0"
    assert isinstance(n, int), "Количество групп не может быть выражено дробным числом"

    output_list = []

    i = 0
    temp_n = n

    length = len(input_obj)

    while temp_n > 0:
        # определяем размер очередной группы
        part_length = math.ceil(length / temp_n) 

        # логика переноса элементов в новый список
        # отщипываем в новый массив часть исходного объекта по маске [i: i + part_length]
        output_list.append(input_obj[i: i + part_length])

        # обновляем кол-во оставшихся элементов в исходном объекте
        length -= part_length
        # обновляем 'курсор'
        i += part_length
        # обновляем кол-во групп, для которых еще не определено кол-во элементов
        temp_n -= 1

    return output_list

### Тесты

In [41]:
a = [[1,'a','b'], [2, 'c', 'd'], [3, 'e', 'f']]
assert split_list(a, 2) == [[[1, 'a', 'b'], [2, 'c', 'd']], [[3, 'e', 'f']]]

In [42]:
a = [1,2,3,4,5,6,7,8,9]
assert split_list(a, 4) == [[1,2,3], [4,5], [6,7], [8,9]]

In [43]:
a = [1,2,3,4,5,6,7,8,9,10]
assert split_list(a, 4) == [[1,2,3], [4,5, 6], [7,8], [9, 10]]

In [44]:
# дополнительные тесты
try:
    a = []
    split_list(a, 2)
except Exception as ex:
    print(ex)

Исходный объект пустой


In [45]:
# дополнительные тесты
try:
    a = [1,2,3,4,5,6,7,8,9]
    split_list(a, 0)
except Exception as ex:
    print(ex)

Количество групп не может быть отрицательным числом или 0


In [46]:
# дополнительные тесты
try:
    a = [1,2,3,4,5,6,7,8,9]
    split_list(a, 1.5)
except Exception as ex:
    print(ex)

Количество групп не может быть выражено дробным числом


In [47]:
# дополнительные тесты
a = [1,2,3,4,5,6,7,8,9,10,11]
assert split_list(a, 4) == [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11]]

a = [1,2,3,4,5,6,7,8,9,10,11,12]
assert split_list(a, 5) == [[1, 2, 3], [4, 5, 6], [7, 8], [9, 10], [11, 12]]

a = [1,2,3,4,5,6,7,8,9,10,11,12,13]
assert split_list(a, 4) == [[1, 2, 3, 4], [5, 6, 7], [8, 9, 10], [11, 12, 13]]

a = [[1,2,3], [4,5], [6,7,8,9], [10]]
assert split_list(a, 4) == [[[1, 2, 3]], [[4, 5]], [[6, 7, 8, 9]], [[10]]]

a = [[1,2,3], [4,5], [6,7,8,9], [10]]
assert split_list(a, 3) == [[[1, 2, 3], [4, 5]], [[6, 7, 8, 9]], [[10]]]

a = tuple([2, 3, 4, 5, 6, 7, 8])
assert split_list(a, 4) == [(2, 3), (4, 5), (6, 7), (8,)]

a = np.array([2, 3, 4, 5, 6, 7, 8])
assert all([np.array_equal(arr1, arr2) for arr1, arr2 in zip(split_list(a, 4),
                                                             [np.array([2, 3]), np.array([4, 5]), np.array([6, 7]), np.array([8])])])
